<a href="https://colab.research.google.com/github/boontarikau/type-of-data-in-statistics/blob/main/Type_of_data_on_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data source:
Data is available in CSV format through the link below:

(https://data.sfgov.org/City-Management-and-Ethics/Employee-Compensation/88g8-5mnd)\

Data Provided by: SF Controller's Office italicized text

Dataset Owner: OpenData

The San Francisco Controller's Office maintains a database of the salary and benefits paid to City employees since fiscal year 2013. This data is summarized and presented on the Employee Compensation report from 2013 to 2023 both in fiscal. New data is added on a bi-annual basis when available for each fiscal and calendar year.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_original=pd.read_csv('/content/drive/MyDrive/analysis/stats/Copy of Employee_Compensation (1).csv')

In [ ]:
#over view tha data
df_original.head()

In [ ]:
#show the rows and columns of the data set
df_original.shape

In [ ]:
#find out null value
display(df_original.isnull().sum())

Since, there are only  Department Cod= 2 ,Department=	2, Union Code	=180,Union	=179, Job	= 5 and to compare with 881800 rows. There are less and we can remove them

In [ ]:
#delete null value
df = df_original.dropna()
display(df.shape)

In [ ]:
#type of each columns
df.dtypes

In [ ]:
#data type casting: change data type into numeric data
numeric_columns=['Salaries','Overtime','Other Salaries','Total Salary','Retirement','Health and Dental','Other Benefits','Total Benefits','Total Compensation']
for i in df.columns:
  if i not in numeric_columns:
    df[i]=df[i].astype('object')
df.dtypes

In [ ]:
df.describe()

In [ ]:
#seperate fiscal and calendar to analyze the statistical describe
df_f=df[df['Year Type']=='Fiscal']
df_c=df[df['Year Type']=='Calendar']
print (df_f.shape, df_c.shape)

In [ ]:
df_f.describe()

In [ ]:
#from data above, ath salaries can not be negetive amount
df_f[df['Salaries']<0]

In [ ]:
df_f[df_f['Salaries']<0].min()

In [ ]:
df_f[df_f['Salaries']>250000]

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=df_f['Salaries']).set_title('Box Plot of Salaries', fontsize=15)
plt.show()

In [ ]:
#distribution of salaries data with negative numbers
plt.figure(figsize=(15,8))
sns.distplot(df_f['Salaries'],kde=True,bins=40).set_title(' Histogram of Salaries- with outliers',fontsize=15)
plt.show();

In [ ]:
#remove outlier
def remove_outliers(d):
  for i in d.columns:
    if d[i].dtypes!="object":
      lower_bound=d[i].mean()-3*d[i].std()
      upper_bound=d[i].mean()+3*d[i].std()
      d_I=d[(d[i]>=lower_bound)&(d[i]<=upper_bound)]
      d_O=d[(d[i]<lower_bound)&(d[i]>upper_bound)]
  return d_I

df_f_I=remove_outliers(df_f)
print(df_f_I.shape, df_f.shape)

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(df_f_I['Salaries'],kde=True,bins=40).set_title(' Histogram of Salaries- after removing outliers',fontsize=15)
plt.show();

Feature Engineering

five  years interval

In [ ]:
df_f['Five_years_period'] = ''  # Create a new column with empty values

df_f['Five_years_period'] = 'second_half'  # Set the default value to 'second_half'
df_f.loc[df_f['Year'] <= 2017, 'Five_years_period'] = 'first_half'
df_f.dropna(axis=0)

In [ ]:
display(df_f['Job'].unique())

In [ ]:
print(df_f['Job'].nunique())

In [ ]:
job_segment_mapping = {
    'Community Health': ['Health', 'Medical', 'Nurse', 'Physician', 'Dentist', 'Pharmacy', 'Therapist', 'Epidemiologist', 'Paramedic', 'EMT'],
    'Culture & Recreation': ['Library', 'Museum', 'Park', 'Recreation', 'Cultural', 'Zoo', 'Arts'],
    'General Administration & Finance': ['Clerk', 'Assistant', 'Analyst', 'Manager', 'Director', 'Administrator', 'Accountant', 'Auditor', 'Finance', 'Budget', 'Payroll', 'Secretary', 'Coordinator', 'Officer', 'Specialist', 'Planner', 'Executive', 'Commissioner'],
    'General City Responsibilities': ['City Planning', 'Elections', 'Human Rights', 'Ethics', 'Civil Service', 'Retirement', 'Treasurer', 'Controller', 'Attorney', 'Legal', 'Legislative'],
    'Human Welfare & Neighborhood Development': ['Social Worker', 'Human Services', 'Probation', 'Juvenile', 'Child', 'Youth', 'Aging', 'Disability', 'Homeless', 'Housing', 'Community Development'],
    'Public Protection': ['Police', 'Fire', 'Sheriff', 'Correction', 'Probation', 'Emergency', 'Security', 'Investigation', 'Inspector', 'Forensic'],
    'Public Works, Transportation & Commerce': ['Engineer', 'Transit', 'Transportation', 'Public Works', 'Airport', 'MTA', 'Port', 'Utilities', 'Water', 'Wastewater', 'Street', 'Maintenance', 'Mechanic', 'Operator', 'Planner'],
    'Other': [] # This will be for jobs that don't fit into any of the above categories
}

def assign_job_segment(job_title):
    for segment, keywords in job_segment_mapping.items():
        if segment != 'Other':
            for keyword in keywords:
                if keyword.lower() in job_title.lower():
                    return segment
    return 'Other'

df_f['Job Segment'] = df_f['Job'].apply(assign_job_segment)

# Display the distribution of job segments
display(df_f['Job Segment'].value_counts())

In [ ]:
display(df_f['Job Segment'].unique())

Ploting Graphs

1. Two plots displaying the distribution of a single continuous variable

In [ ]:
# Create Histogram for Total Bebefits
plt.figure(figsize=(15,8))
sns.histplot(df_f['Total Benefits'], bins=30, kde=False)
# Creating Mean and Median Lines
plt.axvline(df_f['Total Benefits'].mean(), color='orange', linestyle='--')
plt.axvline(df_f['Total Benefits'].median(), color='blue', linestyle='-')
# Create and plot Mean and Median Legend
plt.legend({'Mean':df_f['Total Benefits'].mean(),'Median':df_f['Total Benefits'].median()})
# Define Plot Title and Labels
plt.title('Figure 1.1: Distribution of Total Benefits')
plt.xlabel('Total Benefits')
plt.ylabel('Frequency')
plt.show();

In [ ]:
# Create BoxPlot for Total Benefits
plt.figure(figsize=(15,8))
sns.boxplot(x=df_f['Total Benefits']).set_title('Figure 1.2: Box Plot of Total Benefits', fontsize=15)
plt.show()

2.Two plots displaying information about a single categorical variable

In [ ]:
# create unique data set of employee
unique_employee_count_by_Organization = df_f.groupby('Organization Group')['Employee Identifier'].nunique().reset_index(name='Unique Employee Count')
#bar plot for categorical variable
plt.figure(figsize=(30, 8))
fig = sns.barplot(x='Organization Group', y='Unique Employee Count', data=unique_employee_count_by_Organization, palette='magma', order=unique_employee_count_by_Organization.sort_values(by='Unique Employee Count', ascending=False)['Organization Group'])
fig.set_title('Figure 2.1: Number of Unique Employees in Each Organization Group - Fiscal Year', fontsize=20)

plt.show()
unique_employee_count_by_Organization

In [ ]:
df_f_Organization_Group = df_f['Organization Group'].value_counts()
df_f_Organization_Group_per = df_f_Organization_Group/len(df_f)*100
plt.figure(figsize=(10,6))
colors = sns.color_palette("deep")
plt.pie(df_f_Organization_Group_per,labels =df_f_Organization_Group_per.index,colors=colors,autopct ='%1.2f%%')
plt.title('Figure 2.2: The Percentage of Each section in Organization Group')
plt.show()


3.one plot displaying information about both a continuous variable and a categoricalvariable

In [ ]:
# Create Line Graph for Total Salary by Year
plt.figure(figsize=(15, 8))
sns.lineplot(x='Year', y='Total Salary', data= df_f, ci=None)
plt.title('Figure 3.1: Line Graph of Total Salary by Year')
plt.show();

In [ ]:
# Create BoxPlot of Total Salaries and Job Segment
plt.figure(figsize=(15,8))
sns.boxplot(y=df_f['Job Segment'],x=df_f['Total Salary'])
plt.title('Figure 3.2: Boxplot of Total Salaries by Job Segment',fontsize=15)
plt.show();

4.two plots should display information that shows a relationship between two variables

In [ ]:
# scatter plot between salary and retirement ()
plt.figure(figsize=(18,5))
plt.scatter(df_f['Salaries'],df_f['Retirement'], color='blue')
plt.title(f'Scatterplot of Salaries and Retirement- Fiscal year', fontsize=15)
plt.xlabel('Salaries')
plt.ylabel('Retirement')

# Draw regression line
model = LinearRegression()
X = df_f['Salaries'].values.reshape(-1, 1)  # Reshape x to make it a 2D array
y = df_f['Retirement']
model.fit(X, y)

# Plot the regression line
plt.plot(df_f['Salaries'], model.predict(X), color='red', linewidth=2)

# Add regression equation text to the plot
equation_text = f'Regression Equation:\nRetirement = {model.coef_[0]:.2f} * Salaries + {model.intercept_:.2f}'
plt.text(0.5, 0.9 * df_f['Retirement'].max(), equation_text, fontsize=10, color='green')

plt.show()

In [ ]:
# scatter plot between Total salary and Total Benefits ()
plt.figure(figsize=(18,5))
plt.scatter(df_f['Total Salary'],df_f['Total Benefits'], color='blue')
plt.title(f'Scatterplot of Total Salary and Total Benefits- Fiscal year', fontsize=15)
plt.xlabel('Total Salary')
plt.ylabel('Total Benefits')

# Draw regression line
model = LinearRegression()
X = df_f['Total Salary'].values.reshape(-1, 1)  # Reshape x to make it a 2D array
y = df_f['Total Benefits']
model.fit(X, y)

# Plot the regression line
plt.plot(df_f['Total Salary'], model.predict(X), color='red', linewidth=2)

# Add regression equation text to the plot
equation_text = f'Regression Equation:\nTotal Benefits = {model.coef_[0]:.2f} * Total Salary + {model.intercept_:.2f}'
plt.text(0.5, 0.9 * df_f['Total Benefits'].max(), equation_text, fontsize=10, color='green')

plt.show()

In [ ]:
# scatter plot between Retirement and Total Benefits ()
plt.figure(figsize=(18,5))
plt.scatter(df_f['Retirement'],df_f['Total Benefits'], color='blue')
plt.title(f'Scatterplot of Retirement and Total Benefits- Fiscal year', fontsize=15)
plt.xlabel('Retirement')
plt.ylabel('Total Benefits')

# Draw regression line
model = LinearRegression()
X = df_f['Retirement'].values.reshape(-1, 1)  # Reshape x to make it a 2D array
y = df_f['Total Benefits']
model.fit(X, y)

# Plot the regression line
plt.plot(df_f['Retirement'], model.predict(X), color='red', linewidth=2)

# Add regression equation text to the plot
equation_text = f'Regression Equation:\nTotal Benefits = {model.coef_[0]:.2f} * Retirement + {model.intercept_:.2f}'
plt.text(0.5, 0.9 * df_f['Total Benefits'].max(), equation_text, fontsize=10, color='green')

plt.show()

5.One plot should display information about 3 or more variables

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 8))
sns.boxplot(x="Five_years_period", y="Total Salary", hue="Organization Group", data=df_f, palette="Set3")

plt.title("Box Plot of Total Compensation by Year Type and Organization Group")
plt.ylabel("Total Salary")
plt.xlabel("5 Years Period")
plt.xticks(rotation=45)

plt.legend(loc='upper right', title="Organization Group")
plt.show()